# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 3-1. Задача TSP: инкрементальные алгоритмы.

В этой задаче Вам предлагается сравнить алгоритмы Nearest Neighbour и Nearest Insertion в задаче Euclidean TSP.

**Даны:**
* Координаты точек плоскости, являющихся вершинами графа.

**Найти:**
* Перестановку вершин, задающих минимальный по длине гамильтонов цикл в графе.

Сделайте следующее:
* Скачайте файл [`tsp-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/tsp-instances.zip) и разархивируйте из него файлы со входами задачи TSP.
* Реализуйте функции `solve_tsp_nearest_neighbour` и `solve_tsp_nearest_insertion`.
* Запустите функцию `run_all()`, чтобы протестировать свой код и сравнить качество решений, получаемых Nearest Neighbour и Nearest Insertion. Сильно ли они отличаются? Запишите свои качественные выводы в 1-2 предложениях в последней ячейке ipynb-файла.

In [157]:
def read_tsp_instance(filename):
    with open(filename, 'r') as file:
        coordinates = []
        for line in file:
            line = line.strip().lower()
            if line.startswith('dimension'):
                coordinates = [(0,0)] * int(line.split()[-1])
            tokens = line.split()
            if len(tokens) == 3 and tokens[0].isdecimal():
                tokens = line.split()
                coordinates[int(tokens[0])-1] = tuple(map(float, tokens[1:]))
        return coordinates

from math import sqrt

def euclidean_distance(point1, point2):
    return sqrt((point1[0]-point2[0]) ** 2 + (point1[1]-point2[1]) ** 2)

def calculate_tour_length(instance, permutation):
    n = len(permutation)
    return sum(euclidean_distance(instance[permutation[i]], instance[permutation[(i+1) % n]]) for i in range(len(permutation)))

In [158]:
# a = open_v[], b = coord
def closest(ar, b):
    minim = euclidean_distance(ar[0], b)
    min_x = ar[0]
    for x in ar:
        if (euclidean_distance(x, b) < minim):
            minim = euclidean_distance(x, b)
            min_x = x
    return min_x        

In [159]:
def solve_tsp_nearest_neighbour(instance):
    
    min_tsp_tour = []
    open_v = list(instance)
    N = len(instance)
    
    coord = open_v[N-1]
    del open_v[N-1]
    
    min_tsp_tour += [instance.index(coord)]
    
    for i in range(N-1):
        min_dist_coord = closest(open_v, coord)
        
        coord = min_dist_coord
        
        min_tsp_tour += [instance.index(min_dist_coord)]
        
        open_v.remove(min_dist_coord)
    
    # The return value is permutation of vertices that corresponds to a minimal TSP tour
    return min_tsp_tour

In [160]:
def tour_closest(instance, open_v, tour):
    
    place_near_tour = 0
    idx_near_tour = 0
    
    minim = 10 ** 5
        
    for idxe, cord in enumerate(tour[:-1]):            
        for x in open_v:
            d = euclidean_distance(instance[cord], instance[x])
            d -= euclidean_distance(instance[cord], instance[tour[idxe + 1]])
            d += euclidean_distance(instance[tour[idxe + 1]], instance[x])
                
            if (minim > d):
                minim = d
                idx_near_tour = x
                place_near_tour = idxe
        
    return place_near_tour, idx_near_tour

In [161]:
# a = open_v[], b = 0
def closest_idx(instance, ar, b):
    
    min_idx = ar[0]
    minim = euclidean_distance(instance[min_idx], instance[b])
    
    for i in range(len(ar)):
        if (euclidean_distance(instance[ar[i]], instance[b]) < minim):
            minim = euclidean_distance(instance[ar[i]], instance[b])
            min_idx = ar[i]
            
    return min_idx        

In [165]:
def solve_tsp_nearest_insertion(instance):

    N = len(instance)
    open_v = [x for x in range(1, N)]
    
    # first tour
    tour = [0, 0, 0]
    tmp = closest_idx(instance, open_v, 0)
    tour[1] = tmp
    
    del open_v[tmp-1]
    
    for i in range(N - 2):
        place, idx = tour_closest(instance, open_v, tour)
        
        tour.insert(place + 1, idx)
        
        open_v.remove(idx)
    
    del tour[-1]
    
    # The return value is permutation of vertices that corresponds to a minimal TSP tour
    return tour

In [168]:
import time
from os.path import exists

def run_all():
    instance_filenames = ['d198.tsp', 'd493.tsp', 'd657.tsp', 'pr107.tsp', 'pr152.tsp', 'pr439.tsp']
    instance_filenames = ['tsp-instances/' + name for name in instance_filenames]
    for filename in instance_filenames:
        if not exists(filename):
            print('File not found: “{}”. Skipping this instance.'.format(filename))
            continue
        instance = read_tsp_instance(filename)
        print('Solving instance {}…'.format(filename), end='')
        time_start = time.monotonic()
        quality_nn = calculate_tour_length(instance, solve_tsp_nearest_neighbour(instance))
        time_nn = time.monotonic()-time_start
        time_start = time.monotonic()
        quality_ni = calculate_tour_length(instance, solve_tsp_nearest_insertion(instance))
        time_ni = time.monotonic()-time_start
        print(' done in {:.2} seconds with tour length {} using NN and in {:.2} seconds with tour length {} using NI'.format(time_nn, int(quality_nn), time_ni, int(quality_ni)))

In [169]:
run_all()

Solving instance tsp-instances/d198.tsp… done in 0.019 seconds with tour length 18830 using NN and in 2.5 seconds with tour length 17587 using NI
Solving instance tsp-instances/d493.tsp… done in 0.076 seconds with tour length 44160 using NN and in 3.9e+01 seconds with tour length 39982 using NI
Solving instance tsp-instances/d657.tsp… done in 0.15 seconds with tour length 62860 using NN and in 9.3e+01 seconds with tour length 57906 using NI
Solving instance tsp-instances/pr107.tsp… done in 0.0048 seconds with tour length 47464 using NN and in 0.39 seconds with tour length 52587 using NI
Solving instance tsp-instances/pr152.tsp… done in 0.0084 seconds with tour length 85314 using NN and in 1.1 seconds with tour length 87848 using NI
Solving instance tsp-instances/pr439.tsp… done in 0.062 seconds with tour length 131702 using NN and in 2.7e+01 seconds with tour length 130254 using NI


## Выводы
(Опишите в 1-2 предложениях свои наблюдения по результатам запусков.)

NI чаще работает лучше, но иногда гораздо медленнее.